In [118]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:
from amd_bkg_matchids import *

In [120]:
# 1. fetch all unmatched Amadeus hotels
print("Loading Amadeus hotels")
amdh = load_amadeus_from_db()
print(len(amdh))

Loading Amadeus hotels
5080


In [121]:
# 2. fetch all unmatched Booking.com hotels
print("Loading Booking hotels")
bkgh = load_booking()
print(len(bkgh))

Loading Booking hotels
880357


In [104]:
# 3. load existing matches
matches = {}
# matches = load_matches()

# matched_amdids = matches.keys()
# matched_bkgids = set(matches.values())

# 4. Exclude already matched
# amdh = amdh[~amdh.amd_id.isin(matched_amdids)]
# bkgh = bkgh[~bkgh.bkg_id.isin(matched_bkgids)]

# 5. Build geo index
print("Building Geo Index")
geo_index = GeoGridIndex()
for i, hb in bkgh.iterrows():
    if hb['lat'] == 90:
        hb['lat'] = -90.0    
    geo_index.add_point(GeoPoint(hb['lat'], hb['lng'], ref=hb))

Building Geo Index


In [105]:
h = list(amdh[amdh.amd_id == '06af7a90-ad66-4426-a235-c531036e884b'].iterrows())[0][1]

center_point = GeoPoint(h['lat'], h['lng'], ref=h)

radius = 20
nsim_threshold = 0.6
cands = list(geo_index.get_nearest_points(center_point, radius, 'km'))

In [106]:
len(cands)

1169

In [55]:
h


amd_id    f748b379-c15b-4184-9fed-f19be2541c53
lat                                   -27.5437
lng                                    151.923
name                 Altitude Motel Apartments
chain                                AADX PLUS
bkg_id                             1.47678e+06
Name: f748b379-c15b-4184-9fed-f19be2541c53, dtype: object

In [56]:
center_point = GeoPoint(h['lat'], h['lng'], ref=h)


In [122]:
import json
with open('matches_old.json') as f:
    matches = json.load(f)

In [123]:
len(matches)

4358

In [124]:
missing = [x for x in matches.values() if x not in bkgh.bkg_id.values]

In [125]:
len(missing)

17

In [14]:
amdh.bkg_id = amdh.amd_id.apply(lambda i: matches[i] if i in matches else None)

In [107]:
matches_df = amdh.join(bkgh, on='bkg_id', how='inner', rsuffix='_bkg')
matches_df = matches_df[['amd_id', 'bkg_id_bkg',u'name', 'name_bkg', u'chain', 'chain_bkg', 'lat',  u'lng', u'lat_bkg', u'lng_bkg']]
len(matches_df)

0

In [53]:
matches_df.sample(50)

,amd_id,bkg_id_bkg,name,chain,name_bkg,chain_bkg,lat,lng,lat_bkg,lng_bkg
amd_id,,,,,,,,,,
ffcd8228-caca-4a03-ae8e-cf56036db896,ffcd8228-caca-4a03-ae8e-cf56036db896,266852,SPRINGHILL STES BOISE MARRIOTT,SPRINGHILL SUITES,SpringHill Suites Boise,SpringHill Suites,43.66230,-116.33547,43.6624,-116.3351
7ce770c3-e696-4b6a-90eb-c3930aec1ab9,7ce770c3-e696-4b6a-90eb-c3930aec1ab9,518647,EXTENDED STAY AMERICA CARNEGIE,EXTENDED STAY,Extended Stay America - Pittsburgh - Carnegie,Extended Stay America,40.41639,-80.08813,40.4173,-80.0886
3950fd36-6321-4091-ada1-c4485bcf47d2,3950fd36-6321-4091-ada1-c4485bcf47d2,171164,SPRINGHILL STES MIDLA MARRIOTT,SPRINGHILL SUITES,SpringHill Suites Midland,SpringHill Suites,43.65956,-84.23589,43.6577,-84.2321
30fe7028-6345-415e-9aab-c440463fe5fa,30fe7028-6345-415e-9aab-c440463fe5fa,267456,TOWNEPLACE STES DULLE MARRIOTT,TOWNEPLACE SUITES,TownePlace Suites Dulles Airport,TownePlace Suites by Marriott,38.98690,-77.42900,38.9868,-77.4296
4e7b10d5-70de-4f07-9cb1-6882c03f1fac,4e7b10d5-70de-4f07-9cb1-6882c03f1fac,267913,SPRINGHILL STES WILLI MARRIOTT,SPRINGHILL SUITES,SpringHill Suites by Marriott Williamsburg,SpringHill Suites,37.29334,-76.72463,37.2937,-76.7271
d90a8112-68ef-4ca7-bc56-db5eee588f34,d90a8112-68ef-4ca7-bc56-db5eee588f34,189954,SPRINGHILL STES FLAGS MARRIOTT,SPRINGHILL SUITES,SpringHill Suites by Marriott Flagstaff,SpringHill Suites,35.17761,-111.66311,35.1777,-111.6636
1ff37457-df1d-4b12-99d5-ca2b01a65c68,1ff37457-df1d-4b12-99d5-ca2b01a65c68,517079,EXTENDED STAY AMERICA GREAT FA,EXTENDED STAY,Extended Stay America - Great Falls - Missouri...,Extended Stay America,47.49596,-111.30741,47.4963,-111.3078
3e17863e-3b11-4718-b325-9beecf920173,3e17863e-3b11-4718-b325-9beecf920173,237871,STUDIOS 2 LET SERVICED APARTMENTS,FASTBOOKING,Studios2Let,None,51.52685,-0.12762,51.5270,-0.1276
12c7eb67-0396-4458-ad5a-4ee7c9ed95e3,12c7eb67-0396-4458-ad5a-4ee7c9ed95e3,267622,TOWNEPLACE STES QUANT MARRIOTT,TOWNEPLACE SUITES,TownePlace Suites Stafford,TownePlace Suites by Marriott,38.45573,-77.40621,38.4555,-77.4055
